<a href="https://colab.research.google.com/github/arefrazavi/nlp/blob/main/classification_tasks/text_classification_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Required Packages

In [ ]:
import os

# Disable asynchronously kernel launches which is useful for debugging.
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

! pip install datasets transformers numpy

     |████████████████████████████████| 262 kB 5.3 MB/s 
     |████████████████████████████████| 2.5 MB 43.5 MB/s 
     |████████████████████████████████| 243 kB 45.8 MB/s 
     |████████████████████████████████| 118 kB 48.7 MB/s 
     |████████████████████████████████| 43 kB 1.4 MB/s 
     |████████████████████████████████| 895 kB 36.6 MB/s 
     |████████████████████████████████| 3.3 MB 33.9 MB/s 


# Gather Data

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("imdb")

# Each row has following columns:
# text: a string feature.
# label: a classification label, with possible values including neg (0), pos (1).
raw_datasets

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

# Prepare Data

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    # The maximum sequence length in BERT model is 512.
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Select a smaller set from each dataset to have faster training for debugging.
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(2000))
test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(2000))
unsupervised_dataset = tokenized_datasets["unsupervised"].shuffle(seed=42).select(range(100))

print(f"\nTraining Sample: \n Label: {train_dataset[0]['label']}, \n Text: {train_dataset[0]['text']}")



Training Sample: 
 Label: 0, 
 Text: This is your typical Priyadarshan movie--a bunch of loony characters out on some silly mission. His signature climax has the entire cast of the film coming together and fighting each other in some crazy moshpit over hidden money. Whether it is a winning lottery ticket in Malamaal Weekly, black money in Hera Pheri, "kodokoo" in Phir Hera Pheri, etc., etc., the director is becoming ridiculously predictable. Don't get me wrong; as clichéd and preposterous his movies may be, I usually end up enjoying the comedy. However, in most his previous movies there has actually been some good humor, (Hungama and Hera Pheri being noteworthy ones). Now, the hilarity of his films is fading as he is using the same formula over and over again.<br /><br />Songs are good. Tanushree Datta looks awesome. Rajpal Yadav is irritating, and Tusshar is not a whole lot better. Kunal Khemu is OK, and Sharman Joshi is the best.


# Define and Train Model

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# Pretrain our BERT model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

# Create a trainer from our pre-trained model to fine tune it.
training_args = TrainingArguments("test_trainer")
trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset
)

# Fine tune our model
trainer.train()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Step,Training Loss
500,0.374300


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=750, training_loss=0.2851548741658529, metrics={'train_runtime': 1209.9564, 'train_samples_per_second': 4.959, 'train_steps_per_second': 0.62, 'total_flos': 1996403281920000.0, 'train_loss': 0.2851548741658529, 'epoch': 3.0})

Evaluate Model

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 8


{'eval_accuracy': 0.8905,
 'eval_loss': 0.5139355659484863,
 'eval_runtime': 140.0499,
 'eval_samples_per_second': 14.281,
 'eval_steps_per_second': 1.785}

# Prediction

In [ ]:
import numpy as np
import csv

# Removing the `label` columns because it contains -1 and Trainer won't like that.
predict_dataset = unsupervised_dataset.remove_columns("label")

predictions = trainer.predict(predict_dataset, metric_key_prefix="predict").predictions
predictions = np.argmax(predictions, axis=1)


label_list = raw_datasets["train"].features["label"].names
output_predict_file = os.path.join(training_args.output_dir, f"predict_results.csv")
text_label_predictions = []
for index, prediction_label in enumerate(predictions):
    label_name = label_list[prediction_label]
    text_label_predictions.append({
        "text": predict_dataset['text'][index],
        "label": label_name
    })

with open(output_predict_file, "w") as file:
    writer = csv.DictWriter(file, fieldnames=["text", "label"])
    writer.writeheader()
    writer.writerows(text_label_predictions)
   
text_label_predictions

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 100
  Batch size = 8


[{'label': 'pos',
  'text': 'a great soundtrack by Tom Waits, the great director Francis Ford Coppola, the sceneries, and the most beautiful actress in the world ever (Nastassia Kinsky), make a not-very-unusual story into a movie that must figure in the History of Movies. A great love story.'},
 {'label': 'pos',
  'text': "I know many find that the sequels are disappointing and a real let down for the original film however i found this sequel fantastic! The animation is superb, and you can tell there was a huge amount of effort into making the scenery as great as the first one. It's a great in between story about from when Bambi's mother dies and he is reunited with his father the Great Prince. Bambi and his friends Thumper, Flower and Faline are all there and are wonderfully voiced. It was very funny yet it was also a strong story showing Bambi getting to know his father, very emotional at some points! I really really enjoyed it!! I hope that you give Bambi 2 the chance to prove its q